In [47]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3 as db

In [2]:
URL = "https://tiendapanini.com.mx/coleccionables/item-3/"
page = requests.get(URL)

store = BeautifulSoup(page.content, "html.parser")

In [3]:
mangas = {}

# Webscrapping

In [4]:
def get_products_in_grid(store_page, mangas):
    products = store.find_all(class_="product-item-link", href=True)
    for product in products:
        mangas[product.text] = product["href"]
    return mangas

In [5]:
def next_page(store,i):
    next_page = f"https://tiendapanini.com.mx/coleccionables/item-3?p={i}"
    page = requests.get(next_page)
    store = BeautifulSoup(page.content, "html.parser")
    spans = store.find_all(class_="toolbar-number")
    first_in_pag = spans[0].text
    if first_in_pag == "1":
        return None
    return store

In [6]:
for i in range(300):
    if i%20==0:
        print(f"Pag number #{i}")
    mangas = get_products_in_grid(store, mangas)
    store = next_page(store, i + 2)
    if store is None:
        break

Pag number #0
Pag number #20
Pag number #40
Pag number #60
Pag number #80
Pag number #100
Pag number #120
Pag number #140
Pag number #160
Pag number #180
Pag number #200
Pag number #220
Pag number #240
Pag number #260
Pag number #280


In [7]:
def filter_by_nonumerated(titulos_tienda):
    sin_numero = titulos_tienda["Numero"].isnull()
    no_clasificados = titulos_tienda[sin_numero].copy()
    return no_clasificados

# First Check

In [8]:
len(mangas)

3429

In [9]:
mangas["NANA N.5"]

'https://tiendapanini.com.mx/nana-n5'

# Create Dataframe

In [10]:
mangas_df = pd.DataFrame(mangas.items(),columns=["Titulo","Link"])

## Split by numbers

In [11]:
titulos_tienda = pd.DataFrame()
titulos_tienda[["Titulo","Numero"]] = mangas_df["Titulo"].str.split(" N.", expand=True, regex=False, n = 1)

In [12]:
titulos_tienda

,Titulo,Numero
0,"JUNJI ITO, LO MEJOR DE LO MEJOR",1
1,NARUTO GOLD EDITION,1
2,NANA,5
3,NEGIMA! - BOXSET,None
4,RE: ZERO (CHAPTER FOUR),3
...,...,...
3424,BUNGO STRAY DOGS,5
3425,BUNGO STRAY DOGS,6
3426,BUNGO STRAY DOGS,4
3427,BUNGO STRAY DOGS,2


# Non numerated: Boxsets

In [13]:
no_clasificados = filter_by_nonumerated(titulos_tienda)
no_clasificados

,Titulo,Numero
3,NEGIMA! - BOXSET,None
34,DEADPOOL SAMURAI 2,None
51,ONE PUNCH MAN ENCICLOPEDIA DE HÉROES,None
52,POKÉMON DIAMOND & PEARL PLATINUM,None
117,Hellbound #1,None
...,...,...
3314,STAR WARS LEGENDS: DARTH MAUL: SENTENCIA DE MU...,None
3315,STAR WARS LEGENDS: LA TRILOGIA DE THRAWN 2: EL...,None
3318,VAGABOND PACK 3,None
3330,NEGIMA 8,None


In [14]:
boxset = no_clasificados["Titulo"].str.contains("BOXSET")
boxsets = no_clasificados[boxset]

boxsets

,Titulo,Numero
3,NEGIMA! - BOXSET,None
159,AOT BEFORE THE FALL - BOXSET,None
164,KUROKO NO BASKET - BOXSET,None
199,BLAME! LUXURY - BOXSET,None
366,BOXSET Sailor V / Sailor Moon Short Stories,None
551,ZELDA BOXSET,None
588,STROBE EDGE BOXSET,None
679,SAINT SEYA GAIDEN BOXSET,None
687,SAKAMOTO DESUGA BOXSET,None
697,ROSARIO VAMPIRE BOXSET,None


In [15]:
vol_box = boxsets[boxsets["Titulo"].str.contains(r"(?:(?:2 -)+ (?:BOXSET))|(?: (?:BOXSET) (?:ED.|[1-2])+)")].copy()

vol_box["Numero"] = vol_box["Titulo"].str.extract(r'(\d+)')
vol_box["Titulo"] = vol_box["Titulo"].str.replace(r"(?: (?:2 -))|(?: (?:ED.|[1-2])+)","")
vol_box

C:\Users\randyc\AppData\Local\Temp\ipykernel_22440\2493613513.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  vol_box["Titulo"] = vol_box["Titulo"].str.replace(r"(?: (?:2 -))|(?: (?:ED.|[1-2])+)","")


,Titulo,Numero
863,MAGI BOXSET,2
866,MAGI BOXSET,1
926,GANTZ BOXSET,2
928,GANTZ BOXSET,1
2470,FAIRY TAIL BOXSET,1
2512,DRAGON BALL BOXSET,2
2514,DRAGON BALL BOXSET,1
2623,BLEACH BOXSET,1
2626,BLEACH BOXSET,2


In [16]:
boxsets.loc[vol_box.index,:] = vol_box.copy()
boxsets.fillna("Unico",inplace=True)
boxsets

C:\Users\randyc\AppData\Local\Temp\ipykernel_22440\1240298908.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxsets.loc[vol_box.index,:] = vol_box.copy()
C:\Users\randyc\AppData\Local\Temp\ipykernel_22440\1240298908.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxsets.fillna("Unico",inplace=True)


,Titulo,Numero
3,NEGIMA! - BOXSET,Unico
159,AOT BEFORE THE FALL - BOXSET,Unico
164,KUROKO NO BASKET - BOXSET,Unico
199,BLAME! LUXURY - BOXSET,Unico
366,BOXSET Sailor V / Sailor Moon Short Stories,Unico
551,ZELDA BOXSET,Unico
588,STROBE EDGE BOXSET,Unico
679,SAINT SEYA GAIDEN BOXSET,Unico
687,SAKAMOTO DESUGA BOXSET,Unico
697,ROSARIO VAMPIRE BOXSET,Unico


In [17]:
titulos_tienda.loc[boxsets.index,:] = boxsets
titulos_tienda

,Titulo,Numero
0,"JUNJI ITO, LO MEJOR DE LO MEJOR",1
1,NARUTO GOLD EDITION,1
2,NANA,5
3,NEGIMA! - BOXSET,Unico
4,RE: ZERO (CHAPTER FOUR),3
...,...,...
3424,BUNGO STRAY DOGS,5
3425,BUNGO STRAY DOGS,6
3426,BUNGO STRAY DOGS,4
3427,BUNGO STRAY DOGS,2


## Packs

In [18]:
no_clasificados = filter_by_nonumerated(titulos_tienda)
no_clasificados

,Titulo,Numero
34,DEADPOOL SAMURAI 2,None
51,ONE PUNCH MAN ENCICLOPEDIA DE HÉROES,None
52,POKÉMON DIAMOND & PEARL PLATINUM,None
117,Hellbound #1,None
130,SWSH10 ASTRAL RADIANCE BST ES,None
...,...,...
3313,STAR WARS LEGENDS: LA TRILOGIA DE THRAWN 1: HE...,None
3314,STAR WARS LEGENDS: DARTH MAUL: SENTENCIA DE MU...,None
3315,STAR WARS LEGENDS: LA TRILOGIA DE THRAWN 2: EL...,None
3318,VAGABOND PACK 3,None


In [19]:
no_clasificados["Titulo"].str.contains('|'.join(["PACK","BOX"]))
packs = no_clasificados[no_clasificados["Titulo"].str.contains('|'.join(["PACK","BOX"]))].copy()
packs

,Titulo,Numero
167,CITY HUNTER - PACK,None
235,YUUNA SAN - PACK,None
260,WOLF CHILDREN BOX,None
473,PLUNDERER PACK 3,None
485,PLATINUM END PACK 3,None
682,SAMURAI 7 PACK,None
886,KILL LA KILL BOX,None
1407,JAMES BOND 007 TPB- PACK,None
1418,JAGAAAAAAN! PACK 3,None
1573,FAIRY TAIL 100 YEARS QUEST - PACK,None


In [20]:
numerated_packs = packs[packs["Titulo"].str.contains("(?:(?:PACK.[1-9])+)")].copy()
numerated_packs

,Titulo,Numero
473,PLUNDERER PACK 3,None
485,PLATINUM END PACK 3,None
1418,JAGAAAAAAN! PACK 3,None
1666,ATOM: THE BEGINNING PACK 3,None
1720,MADE IN ABYSS PACK 3,None
1778,ULTRAMAN PACK 3,None
1795,TENKU SHINPAN PACK 3,None
1813,LA MAESTRA DE LAS BROMAS: TAKAGI - SAN PACK 3,None
1963,MOB PSYCHO 100 PACK 3,None
2768,AJIN PACK 3,None


In [21]:
numerated_packs["Numero"] = numerated_packs["Titulo"].str.extract(r'(\d+)') 
numerated_packs["Titulo"] = numerated_packs["Titulo"].str.replace(r"(?:(?:PACK.[1-9])+)","") + " PACK"
numerated_packs

C:\Users\randyc\AppData\Local\Temp\ipykernel_22440\2111066704.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  numerated_packs["Titulo"] = numerated_packs["Titulo"].str.replace(r"(?:(?:PACK.[1-9])+)","") + " PACK"


,Titulo,Numero
473,PLUNDERER PACK,3
485,PLATINUM END PACK,3
1418,JAGAAAAAAN! PACK,3
1666,ATOM: THE BEGINNING PACK,3
1720,MADE IN ABYSS PACK,3
1778,ULTRAMAN PACK,3
1795,TENKU SHINPAN PACK,3
1813,LA MAESTRA DE LAS BROMAS: TAKAGI - SAN PACK,3
1963,MOB PSYCHO 100 PACK,100
2768,AJIN PACK,3


In [22]:
packs.loc[numerated_packs.index] = numerated_packs
packs.fillna("Unico",inplace=True)
packs

,Titulo,Numero
167,CITY HUNTER - PACK,Unico
235,YUUNA SAN - PACK,Unico
260,WOLF CHILDREN BOX,Unico
473,PLUNDERER PACK,3
485,PLATINUM END PACK,3
682,SAMURAI 7 PACK,Unico
886,KILL LA KILL BOX,Unico
1407,JAMES BOND 007 TPB- PACK,Unico
1418,JAGAAAAAAN! PACK,3
1573,FAIRY TAIL 100 YEARS QUEST - PACK,Unico


In [23]:
titulos_tienda.loc[packs.index,:] = packs
titulos_tienda

,Titulo,Numero
0,"JUNJI ITO, LO MEJOR DE LO MEJOR",1
1,NARUTO GOLD EDITION,1
2,NANA,5
3,NEGIMA! - BOXSET,Unico
4,RE: ZERO (CHAPTER FOUR),3
...,...,...
3424,BUNGO STRAY DOGS,5
3425,BUNGO STRAY DOGS,6
3426,BUNGO STRAY DOGS,4
3427,BUNGO STRAY DOGS,2


## Others

In [24]:
no_clasificados = filter_by_nonumerated(titulos_tienda)
no_clasificados

,Titulo,Numero
34,DEADPOOL SAMURAI 2,None
51,ONE PUNCH MAN ENCICLOPEDIA DE HÉROES,None
52,POKÉMON DIAMOND & PEARL PLATINUM,None
117,Hellbound #1,None
130,SWSH10 ASTRAL RADIANCE BST ES,None
...,...,...
3312,STAR WARS LEGENDS: LA TRILOGIA DE THRAWN 3: LA...,None
3313,STAR WARS LEGENDS: LA TRILOGIA DE THRAWN 1: HE...,None
3314,STAR WARS LEGENDS: DARTH MAUL: SENTENCIA DE MU...,None
3315,STAR WARS LEGENDS: LA TRILOGIA DE THRAWN 2: EL...,None


In [25]:
no_clasificados = no_clasificados[no_clasificados["Titulo"].str.contains("#")]
no_clasificados = no_clasificados["Titulo"].str.split(" #",expand=True)
no_clasificados.columns = ["Titulo", "Numero"]
no_clasificados

,Titulo,Numero
117,Hellbound,1
173,Lady Snowblood,1
176,Death Note Black Edition,1
179,Belle,1
1655,Atom: The Beginning,14
1839,Bungo Stray Dogs,14
2550,My Hero Academia,31
2885,Golden Kamui,23
2918,Jujutsu Kaisen,4
2921,Jujutsu Kaisen,3


In [26]:
titulos_tienda.loc[no_clasificados.index,:] = no_clasificados
titulos_tienda

,Titulo,Numero
0,"JUNJI ITO, LO MEJOR DE LO MEJOR",1
1,NARUTO GOLD EDITION,1
2,NANA,5
3,NEGIMA! - BOXSET,Unico
4,RE: ZERO (CHAPTER FOUR),3
...,...,...
3424,BUNGO STRAY DOGS,5
3425,BUNGO STRAY DOGS,6
3426,BUNGO STRAY DOGS,4
3427,BUNGO STRAY DOGS,2


In [27]:
no_clasificados = filter_by_nonumerated(titulos_tienda)
no_clasificados.sort_values("Titulo").head(40)

,Titulo,Numero
2548,20TH CENTURY BOYS 1 (de 22),None
2540,20TH CENTURY BOYS 10 (de 22),None
2538,20TH CENTURY BOYS 11 (de 22),None
2539,20TH CENTURY BOYS 12 (de 22),None
2537,20TH CENTURY BOYS 13 (de 22),None
2536,20TH CENTURY BOYS 14 (de 22),None
2535,20TH CENTURY BOYS 15 (de 22),None
2534,20TH CENTURY BOYS 16 (de 22),None
2533,20TH CENTURY BOYS 17 (de 22),None
2532,20TH CENTURY BOYS 18 (de 22),None


In [28]:
de_sets = no_clasificados[no_clasificados["Titulo"].str.contains(r"(?:\(de \d+\))")].copy()
de_sets["Titulo"] = de_sets["Titulo"].str.replace(r" (?:\(de \d+\))","", regex=True).copy()
de_sets["Numero"] = de_sets["Titulo"].str.extract(r"(\d+)(?!.*\d)")
de_sets["Titulo"] = de_sets["Titulo"].str.replace(r"(\d+)(?!.*\d)","", regex=True)
de_sets

,Titulo,Numero
253,WARCRAFT MANGA: LEYENDAS,4
254,WARCRAFT MANGA: LEYENDAS,5
255,WARCRAFT MANGA: LEYENDAS,1
256,WARCRAFT MANGA: LEYENDAS,3
257,WARCRAFT MANGA: LEYENDAS,2
1541,GHOST IN THE SHELL,3
1542,GHOST IN THE SHELL,2
1544,GHOST IN THE SHELL,1
2528,20TH CENTURY BOYS,22
2529,20TH CENTURY BOYS,21


In [29]:
titulos_tienda.loc[de_sets.index,:] = de_sets
no_clasificados = filter_by_nonumerated(titulos_tienda)
numerados_diferente = no_clasificados[no_clasificados["Titulo"].str.contains(r"NO. \d+")].copy()
numerados_diferente

,Titulo,Numero
1154,MY HERO ACADEMIA NO. 13,None
1156,MY HERO ACADEMIA NO. 10,None
1157,MY HERO ACADEMIA NO. 12,None
1158,MY HERO ACADEMIA NO. 11,None
1159,MY HERO ACADEMIA NO. 7,None
1160,MY HERO ACADEMIA NO. 9,None
1161,MY HERO ACADEMIA NO. 8,None
1162,MY HERO ACADEMIA NO. 6,None
1163,MY HERO ACADEMIA NO. 3,None
1164,MY HERO ACADEMIA NO. 4,None


In [30]:
numerados_diferente["Numero"] = numerados_diferente["Titulo"].str.extract(r"(\d+)(?!.*\d)")
numerados_diferente["Titulo"] = numerados_diferente["Titulo"].str.replace(r"NO. \d+","", regex=True)
numerados_diferente

,Titulo,Numero
1154,MY HERO ACADEMIA,13
1156,MY HERO ACADEMIA,10
1157,MY HERO ACADEMIA,12
1158,MY HERO ACADEMIA,11
1159,MY HERO ACADEMIA,7
1160,MY HERO ACADEMIA,9
1161,MY HERO ACADEMIA,8
1162,MY HERO ACADEMIA,6
1163,MY HERO ACADEMIA,3
1164,MY HERO ACADEMIA,4


In [31]:
titulos_tienda.loc[numerados_diferente.index,:] = numerados_diferente
titulos_tienda

,Titulo,Numero
0,"JUNJI ITO, LO MEJOR DE LO MEJOR",1
1,NARUTO GOLD EDITION,1
2,NANA,5
3,NEGIMA! - BOXSET,Unico
4,RE: ZERO (CHAPTER FOUR),3
...,...,...
3424,BUNGO STRAY DOGS,5
3425,BUNGO STRAY DOGS,6
3426,BUNGO STRAY DOGS,4
3427,BUNGO STRAY DOGS,2


In [32]:
no_clasificados = filter_by_nonumerated(titulos_tienda)
no_clasificados.sort_values("Titulo")

,Titulo,Numero
3119,20th Century Boys / 21st Century Boys Boxsetset,None
307,A SILENT VOICE,None
304,A SILENT VOICE 5,None
2766,AJIN 1,None
2758,AJIN 10,None
...,...,...
2120,SWORD ART ONLINE PHANTOM BULLET 3,None
130,SWSH10 ASTRAL RADIANCE BST ES,None
568,TERRAFORMARS 12,None
2057,VAMPIRE KNIGHT - Colección Completa,None


In [33]:
numerated = no_clasificados[no_clasificados["Titulo"].str.contains(r"(?! NO\.) (?:[0-9]+)(?!\d+$)(?! |[A-z]|\:|\d)")].copy()
numerated.tail(60)

,Titulo,Numero
1297,LOVE STAGE 5,None
1313,OVERLORD: THE BLOODY VALKYRIE N 10,None
1356,MY LITTLE MOSNTER 6,None
1447,INUYASHA 6,None
1448,INUYASHA 4,None
1476,HUNTER X HUNTER 13,None
1824,STAR WARS MANGA: LOST STARS 2,None
1829,STAR WARS MANGA: LOST STARS 3,None
1972,POKEMON BLACK AND WHITE 9,None
1973,POKEMON BLACK AND WHITE 7,None


In [34]:
numerated["Numero"] = numerated["Titulo"].str.extract(r"(\d+)(?!.*\d)")
numerated["Titulo"] = numerated["Titulo"].str.replace(r"(?! NO\.) (?:[0-9]+)(?!\d+$)(?! |[A-z]|\:|\d)", "", regex=True)
numerated

,Titulo,Numero
34,DEADPOOL SAMURAI,2
147,DEADPOOL SAMURAI,1
304,A SILENT VOICE,5
344,SOUL EATER,16
568,TERRAFORMARS,12
...,...,...
3101,DRAGON BALL SUPER,1
3266,DRAGON BALL PARTWORKS No.,15
3269,DRAGON BALL PARTWORKS No.,16
3271,DRAGON BALL PARTWORKSNo.,14


In [35]:
numerated["Titulo"] = numerated["Titulo"].str.replace(r"( No.| N(?!\w)|No.)","",regex=True)
numerated.tail(60)

,Titulo,Numero
1297,LOVE STAGE,5
1313,OVERLORD: THE BLOODY VALKYRIE,10
1356,MY LITTLE MOSNTER,6
1447,INUYASHA,6
1448,INUYASHA,4
1476,HUNTER X HUNTER,13
1824,STAR WARS MANGA: LOST STARS,2
1829,STAR WARS MANGA: LOST STARS,3
1972,POKEMON BLACK AND WHITE,9
1973,POKEMON BLACK AND WHITE,7


In [36]:
titulos_tienda.loc[numerated.index,:] = numerated
titulos_tienda

,Titulo,Numero
0,"JUNJI ITO, LO MEJOR DE LO MEJOR",1
1,NARUTO GOLD EDITION,1
2,NANA,5
3,NEGIMA! - BOXSET,Unico
4,RE: ZERO (CHAPTER FOUR),3
...,...,...
3424,BUNGO STRAY DOGS,5
3425,BUNGO STRAY DOGS,6
3426,BUNGO STRAY DOGS,4
3427,BUNGO STRAY DOGS,2


In [37]:
no_clasificados = filter_by_nonumerated(titulos_tienda)
no_clasificados.sort_values("Titulo").head(10)

,Titulo,Numero
3119,20th Century Boys / 21st Century Boys Boxsetset,None
307,A SILENT VOICE,None
1273,ALICE 19TH,None
1276,ALL YOU NEED IS KILL,None
1724,ANTES DE LA TORMENTA,None
3226,AOT LOST GIRLS,None
3201,ATAQUE DE LOS TITANES ANSWERS (GUIDE BOOK),None
2711,ATAQUE DE LOS TITANES NO.25,None
1699,Akira Boxset,None
3111,CINEMA PURGATORIO POR ALAN MOORE,None


In [38]:
no_clasificados[no_clasificados["Titulo"].str.contains("BOSXSET| Boxset| TOMOS| Paquete| NOVEL| Colección Completa")]= "Unico"
no_clasificados.query('Numero != Numero')

,Titulo,Numero
51,ONE PUNCH MAN ENCICLOPEDIA DE HÉROES,None
52,POKÉMON DIAMOND & PEARL PLATINUM,None
130,SWSH10 ASTRAL RADIANCE BST ES,None
146,One Piece - Estampas Faltantes,None
307,A SILENT VOICE,None
884,KIMI GA KOI NI OCHIRU / TE ENAMORARÁS,None
1152,MY HERO ACADEMIA NO.16,None
1153,MY HERO ACADEMIA NO.15,None
1155,MY HERO ACADEMIA NO.14,None
1273,ALICE 19TH,None


In [39]:
titulos_tienda.loc[no_clasificados.index,:] = no_clasificados
titulos_tienda

,Titulo,Numero
0,"JUNJI ITO, LO MEJOR DE LO MEJOR",1
1,NARUTO GOLD EDITION,1
2,NANA,5
3,NEGIMA! - BOXSET,Unico
4,RE: ZERO (CHAPTER FOUR),3
...,...,...
3424,BUNGO STRAY DOGS,5
3425,BUNGO STRAY DOGS,6
3426,BUNGO STRAY DOGS,4
3427,BUNGO STRAY DOGS,2


In [40]:
no_clasificados = filter_by_nonumerated(titulos_tienda)
numerated_uppercase = no_clasificados[no_clasificados["Titulo"].str.contains(" NO.\d+")]
final_numerated = numerated_uppercase["Titulo"].str.split(" NO.", expand=True)
no_clasificados.loc[final_numerated.index] = final_numerated
no_clasificados

,Titulo,Numero
51,ONE PUNCH MAN ENCICLOPEDIA DE HÉROES,None
52,POKÉMON DIAMOND & PEARL PLATINUM,None
130,SWSH10 ASTRAL RADIANCE BST ES,None
146,One Piece - Estampas Faltantes,None
307,A SILENT VOICE,None
884,KIMI GA KOI NI OCHIRU / TE ENAMORARÁS,None
1152,MY HERO ACADEMIA,16
1153,MY HERO ACADEMIA,15
1155,MY HERO ACADEMIA,14
1273,ALICE 19TH,None


In [41]:
no_clasificados.fillna("Unico",inplace=True)
titulos_tienda.loc[no_clasificados.index,:] = no_clasificados
titulos_tienda

,Titulo,Numero
0,"JUNJI ITO, LO MEJOR DE LO MEJOR",1
1,NARUTO GOLD EDITION,1
2,NANA,5
3,NEGIMA! - BOXSET,Unico
4,RE: ZERO (CHAPTER FOUR),3
...,...,...
3424,BUNGO STRAY DOGS,5
3425,BUNGO STRAY DOGS,6
3426,BUNGO STRAY DOGS,4
3427,BUNGO STRAY DOGS,2


In [42]:
titulos_tienda[titulos_tienda["Numero"].isnull()]

,Titulo,Numero


In [44]:
titulos_tienda["Link"] = mangas_df["Link"]
titulos_tienda

,Titulo,Numero,Link
0,"JUNJI ITO, LO MEJOR DE LO MEJOR",1,https://tiendapanini.com.mx/junji-ito-lo-mejor...
1,NARUTO GOLD EDITION,1,https://tiendapanini.com.mx/naruto-gold-editio...
2,NANA,5,https://tiendapanini.com.mx/nana-n5
3,NEGIMA! - BOXSET,Unico,https://tiendapanini.com.mx/negima-boxset
4,RE: ZERO (CHAPTER FOUR),3,https://tiendapanini.com.mx/re-zero-chapter-fo...
...,...,...,...
3424,BUNGO STRAY DOGS,5,https://tiendapanini.com.mx/bungo-stray-dogs-n5
3425,BUNGO STRAY DOGS,6,https://tiendapanini.com.mx/bungo-stray-dogs-n6
3426,BUNGO STRAY DOGS,4,https://tiendapanini.com.mx/bungo-stray-dogs-n4
3427,BUNGO STRAY DOGS,2,https://tiendapanini.com.mx/bungo-stray-dogs-n2


# YAY Now let's save it as Sqlite

In [48]:
conn = db.connect('../panini_items.db')
titulos_tienda.to_sql(name='productos', con=conn, if_exists='append', index=False)

3429